# KaibaCorp League of Legends Dashboard project

### Data acquisition

In [4]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [5]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# expires after 2 days, generate new api key on https://developer.riotgames.com/
api_key = 'RGAPI-23b8755e-d999-4001-bba8-81161b5f6604'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [7]:
# get summoner
me = watcher.summoner.by_name(my_region, 'Kelody')

# get match history by puuid
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [8]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11] and match_detail["info"]["gameMode"] == "CLASSIC":
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)

participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [9]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5837146556,1,DaZzA8,TOP,Jax,2/7/3,13,7179,20686,1666,...,12,4,2031,6333,6632,3111,0,0,3340,False
1,EUW1_5837146556,2,The Love Fairy,JUNGLE,Rengar,2/10/4,11,10545,29535,8521,...,4,11,6632,2031,3134,3047,1031,3133,3340,False
2,EUW1_5837146556,3,Stormboltz,MIDDLE,Malzahar,8/9/3,14,19290,24716,1016,...,12,4,3020,6653,2420,4637,3116,0,3340,False
3,EUW1_5837146556,4,SaulokiN,BOTTOM,Draven,2/10/4,12,9273,21112,1403,...,7,4,1055,6673,3006,3508,1036,0,3340,False
4,EUW1_5837146556,5,Arisu,UTILITY,Yuumi,3/5/7,12,6074,8572,8394,...,3,14,3158,3222,3853,6617,0,0,3364,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,EUW1_5833733388,6,InRevolt,TOP,Khazix,0/1/0,10,3770,9135,2348,...,14,4,1055,3142,2031,0,3070,0,3340,False
196,EUW1_5833733388,7,Fegel,UTILITY,Renata,0/2/3,8,2939,4872,262,...,14,4,4642,2003,2055,1028,3158,3851,3364,False
197,EUW1_5833733388,8,The Deus Vult,MIDDLE,Viktor,0/2/0,5,419,2235,0,...,4,12,1052,2033,2010,0,0,0,3340,False
198,EUW1_5833733388,9,Kelody,BOTTOM,Samira,2/4/1,9,5841,8909,947,...,4,7,1055,6673,3047,0,0,0,3340,False


In [10]:
matchesDf

,matchId,gameDuration,gameType,gameVersion
0,EUW1_5837146556,1385,MATCHED_GAME,12.7.433.4138
1,EUW1_5837067809,2093,MATCHED_GAME,12.7.433.4138
2,EUW1_5836710888,1466,MATCHED_GAME,12.7.433.4138
3,EUW1_5836548775,1212,MATCHED_GAME,12.7.433.4138
4,EUW1_5836562559,1866,MATCHED_GAME,12.7.433.4138
5,EUW1_5836494579,2225,MATCHED_GAME,12.7.433.4138
6,EUW1_5836430280,1448,MATCHED_GAME,12.7.433.4138
7,EUW1_5836416262,1575,MATCHED_GAME,12.7.433.4138
8,EUW1_5835357892,1832,MATCHED_GAME,12.7.433.4138
9,EUW1_5835341684,1274,MATCHED_GAME,12.7.433.4138


In [11]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [12]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,killerId,victimId,assistingParticipantIds,bounty,positionX,positionY,level,teamId,monsterType,buildingType,laneType,timestampMIN
0,EUW1_5837146556,ITEM_PURCHASED,5.0,3850.0,2906,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,EUW1_5837146556,ITEM_PURCHASED,5.0,2055.0,2906,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,EUW1_5837146556,ITEM_PURCHASED,5.0,3340.0,3467,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,EUW1_5837146556,ITEM_PURCHASED,6.0,2033.0,4788,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,EUW1_5837146556,ITEM_PURCHASED,6.0,3340.0,5151,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11218,EUW1_5833733388,ITEM_PURCHASED,10.0,6632.0,899201,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
11219,EUW1_5833733388,LEVEL_UP,6.0,NaN,902078,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,15.0
11220,EUW1_5833733388,LEVEL_UP,4.0,NaN,902640,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,15.0
11221,EUW1_5833733388,ELITE_MONSTER_KILL,NaN,NaN,902640,NaN,NaN,2.0,NaN,"[3, 4, 5]",NaN,9866.0,4416.0,NaN,100.0,DRAGON,NaN,NaN,15.0


### Data preparation

In [13]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [14]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [15]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,killerId,victimId,assistingParticipantIds,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5837146556,ITEM_PURCHASED,5.0,3850.0,2906,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,Arisu,Yuumi,NaN,NaN,NaN,NaN,False
1,EUW1_5837146556,ITEM_PURCHASED,5.0,2055.0,2906,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,Arisu,Yuumi,NaN,NaN,NaN,NaN,False
2,EUW1_5837146556,ITEM_PURCHASED,5.0,3340.0,3467,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,Arisu,Yuumi,NaN,NaN,NaN,NaN,False
3,EUW1_5837146556,ITEM_PURCHASED,6.0,2033.0,4788,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,My Little Flori,Gangplank,NaN,NaN,NaN,NaN,False
4,EUW1_5837146556,ITEM_PURCHASED,6.0,3340.0,5151,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,My Little Flori,Gangplank,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11218,EUW1_5833733388,ITEM_PURCHASED,10.0,6632.0,899201,NaN,NaN,NaN,NaN,[],...,NaN,NaN,15.0,Ulrakk,Poppy,NaN,NaN,NaN,NaN,False
11219,EUW1_5833733388,LEVEL_UP,6.0,NaN,902078,NaN,NaN,NaN,NaN,[],...,NaN,NaN,15.0,InRevolt,Khazix,NaN,NaN,NaN,NaN,False
11220,EUW1_5833733388,LEVEL_UP,4.0,NaN,902640,NaN,NaN,NaN,NaN,[],...,NaN,NaN,15.0,SYX Ulquiorra,Jinx,NaN,NaN,NaN,NaN,False
11221,EUW1_5833733388,ELITE_MONSTER_KILL,NaN,NaN,902640,NaN,NaN,2.0,NaN,"[3, 4, 5]",...,NaN,NaN,15.0,NaN,NaN,Bard Ross,Zac,NaN,NaN,False


Initialize the stats dataframe for the player "Kelody"

In [16]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
7,EUW1_5837146556,8
12,EUW1_5837067809,3
26,EUW1_5836710888,7
36,EUW1_5836548775,7
47,EUW1_5836562559,8
57,EUW1_5836494579,8
63,EUW1_5836430280,4
73,EUW1_5836416262,4
81,EUW1_5835357892,2
96,EUW1_5835341684,7


Get all events where player "Kelody" was involved

In [17]:
kelodyEvents = []

for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,killerId,victimId,assistingParticipantIds,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5837146556,ITEM_PURCHASED,8.0,1054.0,23189,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
1,EUW1_5837146556,ITEM_PURCHASED,8.0,2003.0,23189,NaN,NaN,NaN,NaN,[],...,NaN,NaN,0.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
2,EUW1_5837146556,WARD_PLACED,NaN,NaN,62438,YELLOW_TRINKET,8.0,NaN,NaN,[],...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,False
3,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,68449,NaN,NaN,7.0,2.0,"[8, 9, 10]",...,NaN,NaN,1.0,NaN,NaN,Vamousss,Lillia,The Love Fairy,Rengar,True
4,EUW1_5837146556,LEVEL_UP,8.0,NaN,129826,NaN,NaN,NaN,NaN,[],...,NaN,NaN,2.0,Kelody,Akali,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,EUW1_5833733388,CHAMPION_KILL,NaN,NaN,759767,NaN,NaN,9.0,2.0,"[7, 10]",...,NaN,NaN,13.0,NaN,NaN,Kelody,Samira,Bard Ross,Zac,False
1335,EUW1_5833733388,ITEM_PURCHASED,9.0,6673.0,859058,NaN,NaN,NaN,NaN,[],...,NaN,NaN,14.0,Kelody,Samira,NaN,NaN,NaN,NaN,False
1336,EUW1_5833733388,ITEM_PURCHASED,9.0,3047.0,860115,NaN,NaN,NaN,NaN,[],...,NaN,NaN,14.0,Kelody,Samira,NaN,NaN,NaN,NaN,False
1337,EUW1_5833733388,LEVEL_UP,9.0,NaN,887736,NaN,NaN,NaN,NaN,[],...,NaN,NaN,15.0,Kelody,Samira,NaN,NaN,NaN,NaN,False


Check how often the Player died in Early Game (pre 10 mins)

In [18]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
7,EUW1_5837146556,8,0
12,EUW1_5837067809,3,0
26,EUW1_5836710888,7,1
36,EUW1_5836548775,7,1
47,EUW1_5836562559,8,0
57,EUW1_5836494579,8,3
63,EUW1_5836430280,4,5
73,EUW1_5836416262,4,0
81,EUW1_5835357892,2,0
96,EUW1_5835341684,7,4


Check how much CS per Minute the player has 

In [19]:
#get CS per minute
def getCSperMinute(row, name):
    gameDuration = matchesDf.loc[matchesDf["matchId"] == row["matchId"]]["gameDuration"].values[0]/60
    csPerGame = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["totalMinionsKilled"].values[0]
    return round(csPerGame / gameDuration, 2)

In [20]:
kelody["csPerMinute"] = kelody.apply(lambda row: getCSperMinute(row, "Kelody"), axis = 1)
kelody

,matchId,participant,deathsPreMin10,csPerMinute
7,EUW1_5837146556,8,0,4.46
12,EUW1_5837067809,3,0,4.99
26,EUW1_5836710888,7,1,5.77
36,EUW1_5836548775,7,1,8.07
47,EUW1_5836562559,8,0,5.40
57,EUW1_5836494579,8,3,5.23
63,EUW1_5836430280,4,5,3.31
73,EUW1_5836416262,4,0,4.99
81,EUW1_5835357892,2,0,1.08
96,EUW1_5835341684,7,4,1.22


Check if the player was involved in a 1v1

In [21]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,killerId,victimId,assistingParticipantIds,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
22,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,542018,NaN,NaN,8.0,3.0,[],...,NaN,9.0,NaN,NaN,Kelody,Akali,Stormboltz,Malzahar,False,True
24,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,625566,NaN,NaN,3.0,8.0,[],...,NaN,10.0,NaN,NaN,Stormboltz,Malzahar,Kelody,Akali,False,True
29,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,686266,NaN,NaN,8.0,2.0,[],...,NaN,11.0,NaN,NaN,Kelody,Akali,The Love Fairy,Rengar,False,True
40,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,935495,NaN,NaN,8.0,1.0,[],...,NaN,16.0,NaN,NaN,Kelody,Akali,DaZzA8,Jax,False,True
43,EUW1_5837146556,CHAMPION_KILL,NaN,NaN,954554,NaN,NaN,8.0,3.0,[],...,NaN,16.0,NaN,NaN,Kelody,Akali,Stormboltz,Malzahar,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,EUW1_5833827508,CHAMPION_KILL,NaN,NaN,767050,NaN,NaN,9.0,4.0,[],...,NaN,13.0,NaN,NaN,Kelody,Twitch,cuppcakee,Samira,False,True
1170,EUW1_5833827508,CHAMPION_KILL,NaN,NaN,1454628,NaN,NaN,4.0,9.0,[],...,NaN,24.0,NaN,NaN,cuppcakee,Samira,Kelody,Twitch,False,True
1255,EUW1_5833770214,CHAMPION_KILL,NaN,NaN,1757776,NaN,NaN,9.0,4.0,[],...,NaN,29.0,NaN,NaN,Ulore,Ashe,Kelody,Twitch,False,True
1329,EUW1_5833733388,CHAMPION_KILL,NaN,NaN,674627,NaN,NaN,9.0,5.0,[],...,NaN,11.0,NaN,NaN,Kelody,Samira,Rymul,Lux,False,True


Count the amount of 1v1s won and lost

In [22]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)

Count turret kills

In [23]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)

Count elite monster kills

In [24]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

Check if the player died because of a gank

In [25]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

Count deaths to ganks

In [26]:
def countDeathsToGanks(row, name):
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["deathToGank"] == True)
            & (kelodyEventsDf["victimName"] == name)
            & (kelodyEventsDf["timestampMIN"] <= 15)
        ]
    )


kelody["deathsToGanks"] = kelody.apply(
    lambda row: countDeathsToGanks(row, "Kelody"), axis=1
)


Count the amount of wards placed

In [27]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["creatorId"] == participantId)
            & (kelodyEventsDf["type"] == "WARD_PLACED")
        ]
    )


kelody["wardsPlaced"] = kelody.apply(
    lambda row: countWardsPlaced(row, "Kelody"), axis=1
)


Count the amount of wards placed before getting ganked (2 mins before gank)

In [28]:
def countWardsPlacedBeforeGanked(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (
        participantsDf["summonerName"] == name)]["participant"].values[0]
    wardsPlacedTimestamps = kelodyEventsDf.loc[(kelodyEventsDf["matchId"] == row["matchId"]) & (
        kelodyEventsDf["creatorId"] == participantId) & (kelodyEventsDf["type"] == "WARD_PLACED")]["timestampMIN"].values
    ganksTimestamps = kelodyEventsDf.loc[
        (kelodyEventsDf["matchId"] == row["matchId"])
        & (kelodyEventsDf["deathToGank"] == True)
        & (kelodyEventsDf["victimName"] == name)
        & (kelodyEventsDf["timestampMIN"] <= 15)
    ]["timestampMIN"].values
    wardsPlacedBeforeGank = 0
    for timestamp in ganksTimestamps:
        for wardTimestamp in wardsPlacedTimestamps:
            if(timestamp - wardTimestamp <= 2 and timestamp - wardTimestamp > 0):
                wardsPlacedBeforeGank += 1
    return wardsPlacedBeforeGank

In [29]:
kelody["wardsPlacedBeforeGanks"] = kelody.apply(lambda row: countWardsPlacedBeforeGanked(row, "Kelody"), axis=1)

Check the amount of kills and assists the player got before 15 mins

In [31]:
def checkRoamKills(row, name):
    if row["assistingParticipantIds"] and (row["killerName"] == name) and (row["type"] == "CHAMPION_KILL") and (row["timestampMIN"] <= 15):
        return True
    return False

def countRoamKills(row):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & ((kelodyEventsDf["kelodyAssist"] == True) & (kelodyEventsDf["timestampMIN"] <= 15) | (kelodyEventsDf["kelodyRoamKill"] == True))])

In [32]:
kelodyEventsDf["kelodyRoamKill"] = kelodyEventsDf.apply(lambda row: checkRoamKills(row, "Kelody"), axis = 1)
kelody["roamKillsAssists"] = kelody.apply(lambda row: countRoamKills(row), axis=1)

Check if the player won the game

In [33]:
def gameWin(row, name):
    return participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["win"].values[0]

In [34]:
kelody["gameWin"] = kelody.apply(lambda row: gameWin(row, "Kelody"), axis=1)

Calculate the KDA of the player

In [35]:
def getKDA(row, name):
    kdaString = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["kda"].values[0]
    kdaStringArr = kdaString.split("/")
    kdaArr = [int(kdaArrPart) for kdaArrPart in kdaStringArr]
    try:
        kda = (kdaArr[0]+kdaArr[2]) / kdaArr[1]
    except:
        kda = (kdaArr[0]+kdaArr[2]) / 1
    return round(kda, 2)

In [36]:
kelody["kda"] = kelody.apply(lambda row: getKDA(row, "Kelody"), axis=1)

### Final stats

In [37]:
kelody

,matchId,participant,deathsPreMin10,csPerMinute,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced,wardsPlacedBeforeGanks,roamKillsAssists,gameWin,kda
7,EUW1_5837146556,8,0,4.46,5,1,1,0,0,0,1,7,1,8,True,4.75
12,EUW1_5837067809,3,0,4.99,5,1,5,2,0,0,0,17,0,6,True,3.22
26,EUW1_5836710888,7,1,5.77,5,3,1,0,1,0,0,12,0,3,True,4.50
36,EUW1_5836548775,7,1,8.07,3,2,0,1,0,0,0,10,0,1,True,4.00
47,EUW1_5836562559,8,0,5.40,1,0,0,2,1,1,1,14,2,5,True,3.20
57,EUW1_5836494579,8,3,5.23,6,3,0,0,0,0,1,12,1,5,False,3.00
63,EUW1_5836430280,4,5,3.31,1,1,0,0,0,0,0,7,0,6,False,0.57
73,EUW1_5836416262,4,0,4.99,2,0,4,0,1,0,0,8,0,4,True,12.00
81,EUW1_5835357892,2,0,1.08,10,1,1,0,1,0,0,6,0,11,True,3.00
96,EUW1_5835341684,7,4,1.22,4,3,0,0,0,0,2,6,1,2,False,1.11


Export data into .json files

In [38]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
